In [6]:
!pip install scikit-learn

!pip install nltk wordcloud matplotlib
import nltk
# nltk.download('all')

# Download necessary NLTK components
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')  # for Lemm

# وارد کردن کتابخانه‌ها
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


print("All packages are successfully installed!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


All packages are successfully installed!


[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from google.colab import drive
drive.mount('/content/drive')

true_path = '/content/drive/MyDrive/True.csv' #/content/drive/MyDrive/True.csv
fake_path = '/content/drive/MyDrive/Fake.csv' #/content/drive/MyDrive/Fake.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import nltk
import os

# تعیین مسیر داده‌ها به صورت دستی
nltk_data_path = '/root/nltk_data'
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)

nltk.data.path.append(nltk_data_path)

# دانلود مجدد منابع
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
!pip install --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow

In [5]:
!pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.


In [ ]:
# !pip uninstall -y numpy tensorflow tensorflow-text gensim hazm tf-keras numba
# !pip install numpy==1.26.0 tensorflow==2.18.0 tensorflow-text==2.18.0
# !pip show numpy tensorflow tensorflow-text
# !pip install gensim==4.3.3 hazm==0.10.0 numba==0.60.0


# Texts-to-Sequences on Text

In [6]:
# وارد کردن کتابخانه‌های مورد نیاز از Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. خواندن داده‌ها
df_true = pd.read_csv(true_path)
df_fake = pd.read_csv(fake_path)

# 2. برچسب‌گذاری
df_true["label"] = 1
df_fake["label"] = 0
df = pd.concat([df_true, df_fake], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # مخلوط کردن داده‌ها

# 3. انتخاب فقط ستون 'text' (در صورت وجود)
text_column = "text" if "text" in df.columns else df.columns[0]

# =========================================================================================================

# 4. پیش‌پردازش متن با استفاده از Keras Tokenizer
def clean_text(text):
    # 1. تبدیل حروف بزرگ به حروف کوچک
    text = text.lower()

    # 2. حذف فضاهای خالی اضافی
    text = ' '.join(text.split())

    # 3. تجزیه متن به جملات
    sentences = text.split(".")  # ساده‌ترین روش تقسیم جملات با نقطه

    words = []
    stop_words = set(nltk.corpus.stopwords.words('english'))

    for sentence in sentences:
        # 4. توکن‌بندی جملات به کلمات (با استفاده از str.split)
        tokens = sentence.split()

        # 5. حذف اعداد و URLها
        tokens = [word for word in tokens if not word.isdigit() and not word.startswith('http')]

        # 6. حذف علائم نگارشی و ایست‌واژه‌ها
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

        words.extend(tokens)

    return " ".join(words)

# اعمال پیش‌پردازش روی داده‌ها
df["clean_text"] = df[text_column].astype(str).apply(clean_text)

# =========================================================================================================

# 5. تقسیم داده‌ها
X_train_val, X_test, y_train_val, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.176, random_state=42)  # 0.176×0.85 ≈ 0.15

# =========================================================================================================

# 6. استفاده از Keras Tokenizer برای تبدیل متن به دنباله‌های عددی
tokenizer = Tokenizer(num_words=5000)  # تعداد کلمات را محدود به 5000 کلمه می‌کنیم
tokenizer.fit_on_texts(X_train)  # آموزش Tokenizer روی داده‌های آموزشی

X_train_seq = tokenizer.texts_to_sequences(X_train)  # تبدیل متن به دنباله‌های عددی
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# =========================================================================================================

# 7. Padding دنباله‌ها برای یکسان کردن طول دنباله‌ها
max_length = max([len(seq) for seq in X_train_seq])  # طول دنباله‌ها را پیدا می‌کنیم (از طول بزرگترین دنباله استفاده می‌شود)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# =========================================================================================================

# 8. آموزش مدل KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pad, y_train)

# 9. ارزیابی مدل KNN
y_val_pred_knn = knn.predict(X_val_pad)
print("🔷 KNN on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_knn))
print("Precision:", precision_score(y_val, y_val_pred_knn))
print("Recall:", recall_score(y_val, y_val_pred_knn))
print("F1 Score:", f1_score(y_val, y_val_pred_knn))

# =========================================================================================================

# 10. آموزش مدل SVM
svm = SVC()
svm.fit(X_train_pad, y_train)

# 11. ارزیابی مدل SVM
y_val_pred_svm = svm.predict(X_val_pad)
print("\n🔷 SVM on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_svm))
print("Precision:", precision_score(y_val, y_val_pred_svm))
print("Recall:", recall_score(y_val, y_val_pred_svm))
print("F1 Score:", f1_score(y_val, y_val_pred_svm))

# =========================================================================================================

# 12. ارزیابی نهایی روی Test
y_test_pred_knn = knn.predict(X_test_pad)
y_test_pred_svm = svm.predict(X_test_pad)

print("\n📊 Final Comparison on Test Data:")
print("Model\tAccuracy\tPrecision\tRecall\t\tF1 Score")
print(f"KNN\t{accuracy_score(y_test, y_test_pred_knn):.4f}\t\t{precision_score(y_test, y_test_pred_knn):.4f}\t\t{recall_score(y_test, y_test_pred_knn):.4f}\t\t{f1_score(y_test, y_test_pred_knn):.4f}")
print(f"SVM\t{accuracy_score(y_test, y_test_pred_svm):.4f}\t\t{precision_score(y_test, y_test_pred_svm):.4f}\t\t{recall_score(y_test, y_test_pred_svm):.4f}\t\t{f1_score(y_test, y_test_pred_svm):.4f}")


🔷 KNN on Validation:
Accuracy: 0.5938663093642995
Precision: 0.5639078051335777
Recall: 0.6694651741293532
F1 Score: 0.6121694626101791

🔷 SVM on Validation:
Accuracy: 0.6927199642697632
Precision: 0.7112986060161408
Recall: 0.6029228855721394
F1 Score: 0.6526422080107708

📊 Final Comparison on Test Data:
Model	Accuracy	Precision	Recall		F1 Score
KNN	0.5791		0.5493		0.6615		0.6002
SVM	0.6848		0.7002		0.5947		0.6431


# TF-IDF on Text

In [7]:
# 1. خواندن داده‌ها
df_true = pd.read_csv(true_path)
df_fake = pd.read_csv(fake_path)

# 2. برچسب‌گذاری
df_true["label"] = 1
df_fake["label"] = 0
df = pd.concat([df_true, df_fake], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # مخلوط کردن داده‌ها

# 3. انتخاب فقط ستون 'text' (در صورت وجود)
text_column = "text" if "text" in df.columns else df.columns[0]

# =========================================================================================================

# 4. پیش‌پردازش متن
# def clean_text(text):
#     text = text.lower()  # حروف کوچک
#     text = re.sub(r"http\S+", "", text)  # حذف URL
#     text = re.sub(r"\d+", "", text)      # حذف اعداد
#     text = re.sub(r"[^\w\s]", "", text)  # حذف علائم نگارشی
#     tokens = word_tokenize(text)
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word not in stop_words]
#     return " ".join(tokens)

# df["clean_text"] = df[text_column].astype(str).apply(clean_text)

# # پیش‌پردازش متن با استفاده از str.split() به جای word_tokenize
# def clean_text(text):
#     text = text.lower()  # حروف کوچک
#     text = re.sub(r"http\S+", "", text)  # حذف URL
#     text = re.sub(r"\d+", "", text)      # حذف اعداد
#     text = re.sub(r"[^\w\s]", "", text)  # حذف علائم نگارشی
#     tokens = text.split()  # استفاده از str.split() برای توکن‌بندی
#     stop_words = set(stopwords.words('english'))
#     tokens = [word for word in tokens if word not in stop_words]
#     return " ".join(tokens)

# # اعمال پیش‌پردازش روی داده‌ها
# df["clean_text"] = df[text_column].astype(str).apply(clean_text)

# پیش‌پردازش متن با استفاده از NLTK
# def clean_text(text):
#     # 1. تبدیل حروف بزرگ به حروف کوچک
#     text = text.lower()

#     # 2. حذف فضاهای خالی اضافی
#     text = ' '.join(text.split())

#     # 3. تجزیه متن به جملات
#     sentences = nltk.sent_tokenize(text)

#     # 4. توکن‌بندی جملات به کلمات و انجام سایر پردازش‌ها
#     words = []
#     stop_words = set(nltk.corpus.stopwords.words('english'))

#     for sentence in sentences:
#         # 5. توکن‌بندی جملات به کلمات
#         tokens = nltk.word_tokenize(sentence)

#         # 6. حذف اعداد و URLها
#         tokens = [word for word in tokens if not word.isdigit() and not word.startswith('http')]

#         # 7. حذف علائم نگارشی و ایست‌واژه‌ها
#         tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

#         words.extend(tokens)

#     return " ".join(words)

# # اعمال پیش‌پردازش روی داده‌ها
# df["clean_text"] = df[text_column].astype(str).apply(clean_text)

def clean_text(text):
    # 1. تبدیل حروف بزرگ به حروف کوچک
    text = text.lower()

    # 2. حذف فضاهای خالی اضافی
    text = ' '.join(text.split())

    # 3. تجزیه متن به جملات
    sentences = text.split(".")  # ساده‌ترین روش تقسیم جملات با نقطه

    words = []
    stop_words = set(nltk.corpus.stopwords.words('english'))

    for sentence in sentences:
        # 4. توکن‌بندی جملات به کلمات (با استفاده از str.split)
        tokens = sentence.split()

        # 5. حذف اعداد و URLها
        tokens = [word for word in tokens if not word.isdigit() and not word.startswith('http')]

        # 6. حذف علائم نگارشی و ایست‌واژه‌ها
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

        words.extend(tokens)

    return " ".join(words)

# اعمال پیش‌پردازش روی داده‌ها
df["clean_text"] = df[text_column].astype(str).apply(clean_text)

# =========================================================================================================

# 5. تقسیم داده‌ها
X_train_val, X_test, y_train_val, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.176, random_state=42)  # 0.176×0.85 ≈ 0.15

# 6. بردار‌سازی متن
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)

# 7. آموزش مدل KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_vec, y_train)

# 8. ارزیابی مدل KNN
y_val_pred_knn = knn.predict(X_val_vec)
print("🔷 KNN on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_knn))
print("Precision:", precision_score(y_val, y_val_pred_knn))
print("Recall:", recall_score(y_val, y_val_pred_knn))
print("F1 Score:", f1_score(y_val, y_val_pred_knn))

# 9. آموزش مدل SVM
svm = SVC()
svm.fit(X_train_vec, y_train)

# 10. ارزیابی مدل SVM
y_val_pred_svm = svm.predict(X_val_vec)
print("\n🔷 SVM on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_svm))
print("Precision:", precision_score(y_val, y_val_pred_svm))
print("Recall:", recall_score(y_val, y_val_pred_svm))
print("F1 Score:", f1_score(y_val, y_val_pred_svm))

# 11. ارزیابی نهایی روی Test
y_test_pred_knn = knn.predict(X_test_vec)
y_test_pred_svm = svm.predict(X_test_vec)

print("\n📊 Final Comparison on Test Data:")
print("Model\tAccuracy\tPrecision\tRecall\t\tF1 Score")
print(f"KNN\t{accuracy_score(y_test, y_test_pred_knn):.4f}\t\t{precision_score(y_test, y_test_pred_knn):.4f}\t\t{recall_score(y_test, y_test_pred_knn):.4f}\t\t{f1_score(y_test, y_test_pred_knn):.4f}")
print(f"SVM\t{accuracy_score(y_test, y_test_pred_svm):.4f}\t\t{precision_score(y_test, y_test_pred_svm):.4f}\t\t{recall_score(y_test, y_test_pred_svm):.4f}\t\t{f1_score(y_test, y_test_pred_svm):.4f}")


🔷 KNN on Validation:
Accuracy: 0.6592228673514962
Precision: 0.9569569569569569
Recall: 0.2985633978763273
F1 Score: 0.4551297310164247

🔷 SVM on Validation:
Accuracy: 0.9852612773559625
Precision: 0.9816827072337784
Recall: 0.9875078076202374
F1 Score: 0.9845866417561887

📊 Final Comparison on Test Data:
Model	Accuracy	Precision	Recall		F1 Score
KNN	0.6542		0.9451		0.3018		0.4575
SVM	0.9859		0.9829		0.9880		0.9854


In [9]:

# 1. خواندن داده‌ها
df_true = pd.read_csv(true_path)
df_fake = pd.read_csv(fake_path)

# 2. برچسب‌گذاری
df_true["label"] = 1
df_fake["label"] = 0
df = pd.concat([df_true, df_fake], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # مخلوط کردن داده‌ها

# 3. انتخاب فقط ستون 'text' (در صورت وجود)
# text_column = "text" if "text" in df.columns else df.columns[0]

df['text'] = df['title'] + " " + df['text'] + " " + df['subject'] + " " + df['date'].astype(str)  # ترکیب عنوان، متن، موضوع و تاریخ

# =========================================================================================================

# # اعمال پیش‌پردازش روی داده‌ها
# df["clean_text"] = df[text_column].astype(str).apply(clean_text)

def clean_text(text):
    # 1. تبدیل حروف بزرگ به حروف کوچک
    text = text.lower()

    # 2. حذف فضاهای خالی اضافی
    text = ' '.join(text.split())

    # 3. تجزیه متن به جملات
    sentences = text.split(".")  # ساده‌ترین روش تقسیم جملات با نقطه

    words = []
    stop_words = set(nltk.corpus.stopwords.words('english'))

    for sentence in sentences:
        # 4. توکن‌بندی جملات به کلمات (با استفاده از str.split)
        tokens = sentence.split()

        # 5. حذف اعداد و URLها
        tokens = [word for word in tokens if not word.isdigit() and not word.startswith('http')]

        # 6. حذف علائم نگارشی و ایست‌واژه‌ها
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

        words.extend(tokens)

    return " ".join(words)

# اعمال پیش‌پردازش روی داده‌ها
# df["clean_text"] = df[text_column].astype(str).apply(clean_text)

df["clean_text"] = df['text'].astype(str).apply(clean_text)

# =========================================================================================================

# 5. تقسیم داده‌ها
X_train_val, X_test, y_train_val, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.176, random_state=42)  # 0.176×0.85 ≈ 0.15

# 6. بردار‌سازی متن
vectorizer = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.95)
# vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)

# 7. آموزش مدل KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_vec, y_train)

# 8. ارزیابی مدل KNN
y_val_pred_knn = knn.predict(X_val_vec)
print("KNN on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_knn))
print("Precision:", precision_score(y_val, y_val_pred_knn))
print("Recall:", recall_score(y_val, y_val_pred_knn))
print("F1 Score:", f1_score(y_val, y_val_pred_knn))

# 9. آموزش مدل SVM
svm = SVC(kernel='rbf', C=1, gamma='scale')
# svm = SVC()
svm.fit(X_train_vec, y_train)

# 10. ارزیابی مدل SVM
y_val_pred_svm = svm.predict(X_val_vec)
print("\nSVM on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_svm))
print("Precision:", precision_score(y_val, y_val_pred_svm))
print("Recall:", recall_score(y_val, y_val_pred_svm))
print("F1 Score:", f1_score(y_val, y_val_pred_svm))

# 11. ارزیابی نهایی روی Test
y_test_pred_knn = knn.predict(X_test_vec)
y_test_pred_svm = svm.predict(X_test_vec)

print("\nFinal Comparison on Test Data:")
print("Model\tAccuracy\tPrecision\tRecall\t\tF1 Score")
print(f"KNN\t{accuracy_score(y_test, y_test_pred_knn):.4f}\t\t{precision_score(y_test, y_test_pred_knn):.4f}\t\t{recall_score(y_test, y_test_pred_knn):.4f}\t\t{f1_score(y_test, y_test_pred_knn):.4f}")
print(f"SVM\t{accuracy_score(y_test, y_test_pred_svm):.4f}\t\t{precision_score(y_test, y_test_pred_svm):.4f}\t\t{recall_score(y_test, y_test_pred_svm):.4f}\t\t{f1_score(y_test, y_test_pred_svm):.4f}")


KNN on Validation:
Accuracy: 0.8841744826559476
Precision: 0.9376114081996435
Recall: 0.8134859263841633
F1 Score: 0.8711493872143093

SVM on Validation:
Accuracy: 0.9955337204108977
Precision: 0.995666976168369
Recall: 0.9950510361892979
F1 Score: 0.9953589108910891

Final Comparison on Test Data:
Model	Accuracy	Precision	Recall		F1 Score
KNN	0.8937		0.9562		0.8219		0.8840
SVM	0.9952		0.9949		0.9955		0.9952


# Texts-to-Sequences on title, Text, subject, date

In [10]:
# وارد کردن کتابخانه‌های مورد نیاز از Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import nltk

# 1. خواندن داده‌ها
df_true = pd.read_csv(true_path)
df_fake = pd.read_csv(fake_path)

# 2. برچسب‌گذاری
df_true["label"] = 1
df_fake["label"] = 0
df = pd.concat([df_true, df_fake], axis=0)
df = df.sample(frac=1).reset_index(drop=True)  # مخلوط کردن داده‌ها

# 3. انتخاب ستون‌های اضافی برای بهبود دقت مدل
df['text'] = df['title'] + " " + df['text'] + " " + df['subject'] + " " + df['date'].astype(str)  # ترکیب عنوان، متن، موضوع و تاریخ

# =========================================================================================================

# 4. پیش‌پردازش متن با استفاده از Keras Tokenizer
def clean_text(text):
    # 1. تبدیل حروف بزرگ به حروف کوچک
    text = text.lower()

    # 2. حذف فضاهای خالی اضافی
    text = ' '.join(text.split())

    # 3. تجزیه متن به جملات
    sentences = text.split(".")  # ساده‌ترین روش تقسیم جملات با نقطه

    words = []
    stop_words = set(nltk.corpus.stopwords.words('english'))

    for sentence in sentences:
        # 4. توکن‌بندی جملات به کلمات (با استفاده از str.split)
        tokens = sentence.split()

        # 5. حذف اعداد و URLها
        tokens = [word for word in tokens if not word.isdigit() and not word.startswith('http')]

        # 6. حذف علائم نگارشی و ایست‌واژه‌ها
        tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

        words.extend(tokens)

    return " ".join(words)

# اعمال پیش‌پردازش روی داده‌ها
df["clean_text"] = df['text'].astype(str).apply(clean_text)

# =========================================================================================================

# 5. تقسیم داده‌ها
X_train_val, X_test, y_train_val, y_test = train_test_split(df["clean_text"], df["label"], test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.176, random_state=42)  # 0.176×0.85 ≈ 0.15

# =========================================================================================================

# 6. استفاده از Keras Tokenizer برای تبدیل متن به دنباله‌های عددی
tokenizer = Tokenizer(num_words=5000)  # تعداد کلمات را محدود به 5000 کلمه می‌کنیم
tokenizer.fit_on_texts(X_train)  # آموزش Tokenizer روی داده‌های آموزشی

X_train_seq = tokenizer.texts_to_sequences(X_train)  # تبدیل متن به دنباله‌های عددی
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# =========================================================================================================

# 7. Padding دنباله‌ها برای یکسان کردن طول دنباله‌ها
max_length = max([len(seq) for seq in X_train_seq])  # طول دنباله‌ها را پیدا می‌کنیم (از طول بزرگترین دنباله استفاده می‌شود)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# =========================================================================================================

# 8. آموزش مدل KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pad, y_train)

# 9. ارزیابی مدل KNN
y_val_pred_knn = knn.predict(X_val_pad)
print(" _ KNN on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_knn))
print("Precision:", precision_score(y_val, y_val_pred_knn))
print("Recall:", recall_score(y_val, y_val_pred_knn))
print("F1 Score:", f1_score(y_val, y_val_pred_knn))

# =========================================================================================================

# 10. آموزش مدل SVM
svm = SVC()
svm.fit(X_train_pad, y_train)

# 11. ارزیابی مدل SVM
y_val_pred_svm = svm.predict(X_val_pad)
print("\n _ SVM on Validation:")
print("Accuracy:", accuracy_score(y_val, y_val_pred_svm))
print("Precision:", precision_score(y_val, y_val_pred_svm))
print("Recall:", recall_score(y_val, y_val_pred_svm))
print("F1 Score:", f1_score(y_val, y_val_pred_svm))

# =========================================================================================================

# 12. ارزیابی نهایی روی Test
y_test_pred_knn = knn.predict(X_test_pad)
y_test_pred_svm = svm.predict(X_test_pad)

print("\n ____Final Comparison on Test Data:")
print("Model\tAccuracy\tPrecision\tRecall\t\tF1 Score")
print(f"KNN\t{accuracy_score(y_test, y_test_pred_knn):.4f}\t\t{precision_score(y_test, y_test_pred_knn):.4f}\t\t{recall_score(y_test, y_test_pred_knn):.4f}\t\t{f1_score(y_test, y_test_pred_knn):.4f}")
print(f"SVM\t{accuracy_score(y_test, y_test_pred_svm):.4f}\t\t{precision_score(y_test, y_test_pred_svm):.4f}\t\t{recall_score(y_test, y_test_pred_svm):.4f}\t\t{f1_score(y_test, y_test_pred_svm):.4f}")


 _ KNN on Validation:
Accuracy: 0.5813607265148132
Precision: 0.5592137592137593
Recall: 0.6909532483302975
F1 Score: 0.6181423139598045

 _ SVM on Validation:
Accuracy: 0.6601161232693167
Precision: 0.6966161026837806
Recall: 0.5437158469945356
F1 Score: 0.6107416879795396

 ____Final Comparison on Test Data:
Model	Accuracy	Precision	Recall		F1 Score
KNN	0.5791		0.5418		0.6925		0.6079
SVM	0.6775		0.6998		0.5529		0.6177
